In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
invoices = pd.read_excel("6TH-INVOICES.xlsx")

In [45]:
invoices.head()

,Date,No,Invoice No.,A/C,Customer,P,S,B,200g,O.P.,...,1500g,ST,600g,KATIKATI,MANDAZI,DOUGHNUTS,50-50 SLICED,MANDO FAMILY PACK,4 Square,Qty
0,2025-07-06,55250224,632952,55,LUCY WAIRIMU KAIRU (055),0,450,105,192,0,...,0,0,21,0,0,0,0,0,0,768
1,2025-07-06,57250230,632872,57,JAMES MWANGI (057),0,600,300,288,0,...,0,0,128,0,0,0,0,0,60,1376
2,2025-07-06,62250237,632881,62,ANTONY MWANGI (062),15,570,240,480,0,...,0,0,294,0,0,0,0,0,15,1620
3,2025-07-06,63250215,632879,63,NASHON KYULE (063),75,1650,225,360,0,...,0,0,0,0,0,0,0,0,0,2310
4,2025-07-06,82250242,632958,82,PATRICK WAWERU GICHUHI (082),60,450,120,408,0,...,0,0,14,0,0,0,0,0,60,1118


In [46]:
try:
    credit_notes = pd.read_excel("6TH-CREDITS.xlsx")
except:
    credit_notes = pd.DataFrame()
    print("Credit note file not found")

In [47]:
credit_notes.head()

,Date,No,Invoice No.,A/C,Customer,P,S,B,200g,O.P.,...,1500g,ST,600g,KATIKATI,MANDAZI,DOUGHNUTS,50-50 SLICED,MANDO FAMILY PACK,4 Square,Qty
0,2025-07-06,626250200,633763,626,SUSAN NJERI MUCHOKI(626),0,-510,-120,-528,0,...,0,0,-21,0,0,0,0,0,-30,-1209
1,2025-07-06,872250915,633512,872,NAIVAS SUPERMARKET KOMOROK (0872),0,0,0,0,0,...,0,0,0,0,-5,-5,0,0,0,-10
2,2025-07-06,937250350,633522,937,NAIVAS SUPERMARKET KITENGELA (0937),0,0,0,0,0,...,0,0,0,0,-5,-5,0,0,0,-10
3,2025-07-06,1138250269,637599,1138,CLEANSHELF SUPERMARKET - NYAHURURU (1138),0,-75,0,0,0,...,0,0,0,0,0,0,0,0,0,-75
4,2025-07-06,1138250270,637602,1138,CLEANSHELF SUPERMARKET - NYAHURURU (1138),0,0,-45,0,-90,...,0,0,-140,0,0,0,0,0,0,-299


In [48]:
numeric_columns = ['P', 'S', 'B', '200g', 'O.P.', '800g', '1500g', 'ST', '600g', 
                   'KATIKATI', 'MANDAZI', 'DOUGHNUTS', '50-50 SLICED', 'MANDO FAMILY PACK', 
                   '4 Square', 'Qty']

In [49]:
# Function to check if invoice row matches any credit note row for the same A/C
def is_matching_credit_row(invoice_row, credit_notes, numeric_columns):
    ac = invoice_row['A/C']
    # Filter credit notes for the same A/C
    credit_notes_ac = credit_notes[credit_notes['A/C'] == ac]
    if credit_notes_ac.empty:
        return False
    # Compare numeric columns, accounting for negative values in credit notes
    for _, credit_row in credit_notes_ac.iterrows():
        if all(invoice_row[col] == -credit_row[col] for col in numeric_columns):
            return True
    return False

In [50]:
# Flag invoice rows that exactly match credit note rows
if not credit_notes.empty:
    invoices['is_credit'] = invoices.apply(
        lambda row: is_matching_credit_row(row, credit_notes, numeric_columns), axis=1
    )
else:
    invoices['is_credit'] = False

In [52]:
# Filter out matching invoice rows
filtered_df = invoices[~invoices['is_credit']].copy()

In [53]:
# Drop the helper column
filtered_df = filtered_df.drop(columns=['is_credit'])

In [54]:
# Group by A/C to check if any rows remain
remaining_ac = filtered_df['A/C'].unique()

In [55]:
# If no rows remain for an A/C, exclude it
filtered_df = filtered_df[filtered_df['A/C'].isin(remaining_ac)]

In [57]:
# Group remaining invoices by A/C and Customer, summing numeric columns
grouped_df = filtered_df.groupby(['A/C', 'Customer'], as_index=False)[numeric_columns].sum()

In [58]:
# Get the latest date per A/C and Customer
latest_dates = filtered_df.groupby(['A/C', 'Customer'])['Date'].max().reset_index()

In [59]:
# Merge back the latest date
final_df = pd.merge(latest_dates, grouped_df, on=['A/C', 'Customer'], how='right')

In [61]:
# Export to Excel
final_df.to_excel('DATE-6-INVOICES.xlsx', index=False)